In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import joblib 
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge, Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_absolute_error, make_scorer

In [3]:
df = pd.read_csv('data/merged_climate_iom_data.csv')

In [4]:
# Sorting by country and time to ensure proper filling
df.sort_values(by=["country_code", "year", "month"], inplace=True)

columns_to_fill_extended = [
    "AG.LND.FRST.K2", "AG.LND.PRCP.MM", "AG.LND.TOTL.K2", "AG.SRF.TOTL.K2",
    "EG.CFT.ACCS.RU.ZS", "EG.CFT.ACCS.UR.ZS", "EG.CFT.ACCS.ZS", "EG.EGY.PRIM.PP.KD", "EG.ELC.ACCS.ZS", "EG.FEC.RNEW.ZS",
    "EN.GHG.ALL.MT.CE.AR5", "EN.GHG.CH4.AG.MT.CE.AR5", "EN.GHG.CH4.BU.MT.CE.AR5", "EN.GHG.CH4.FE.MT.CE.AR5",
    "EN.GHG.CH4.IC.MT.CE.AR5", "EN.GHG.CH4.MT.CE.AR5", "EN.GHG.CH4.PI.MT.CE.AR5", "EN.GHG.CH4.TR.MT.CE.AR5",
    "EN.GHG.CH4.WA.MT.CE.AR5", "EN.GHG.CO2.BU.MT.CE.AR5", "EN.GHG.CO2.IC.MT.CE.AR5", "EN.GHG.CO2.IP.MT.CE.AR5",
    "EN.GHG.CO2.LU.MT.CE.AR5", "EN.GHG.CO2.MT.CE.AR5", "EN.GHG.CO2.PI.MT.CE.AR5", "EN.GHG.CO2.TR.MT.CE.AR5",
    "EN.GHG.FGAS.IP.MT.CE.AR5", "EN.GHG.N2O.AG.MT.CE.AR5", "EN.GHG.N2O.BU.MT.CE.AR5", "EN.GHG.N2O.FE.MT.CE.AR5",
    "EN.GHG.N2O.IC.MT.CE.AR5", "EN.GHG.N2O.IP.MT.CE.AR5", "EN.GHG.N2O.MT.CE.AR5", "EN.GHG.N2O.PI.MT.CE.AR5",
    "EN.GHG.N2O.TR.MT.CE.AR5", "EN.GHG.N2O.WA.MT.CE.AR5", "ER.FSH.AQUA.MT", "ER.FSH.CAPT.MT", "ER.FSH.PROD.MT",
    "ER.H2O.FWTL.K3", "ER.H2O.INTR.K3", "SP.POP.TOTL"
]

columns_to_fill_extended = [col.lower() for col in columns_to_fill_extended]

# Filtering columns that exist in the dataset
existing_columns = [col for col in columns_to_fill_extended if col in df.columns]

df[existing_columns] = df.groupby("country_code")[existing_columns].transform(lambda x: x.ffill().bfill())

In [5]:
df['total_idp_over_pop'] = (df['internally_displaced_persons'] / df['sp.pop.totl'])*100
df['total_affected_over_pop'] = (df['total_affected'] / df['sp.pop.totl'])*100

In [6]:
weird_values = df[df['total_idp_over_pop'] > 100]

In [7]:
len(weird_values)

8

In [8]:
weird_values[['year','country_name', 'sp.pop.totl','internally_displaced_persons',  'total_idp_over_pop', 'total_affected_over_pop']].describe()

,year,sp.pop.totl,internally_displaced_persons,total_idp_over_pop,total_affected_over_pop
count,8.000000,8.000000e+00,8.000000e+00,8.000000,3.000000
mean,2022.250000,3.535141e+07,4.878471e+07,129.245549,49.949904
std,1.035098,2.027603e+07,3.285272e+07,29.424648,36.602761
min,2021.000000,1.086578e+07,1.114828e+07,102.599951,7.684676
25%,2021.000000,1.086578e+07,1.205802e+07,110.972392,39.383597
50%,2023.000000,5.004279e+07,5.963956e+07,119.177121,71.082518
75%,2023.000000,5.004279e+07,6.685620e+07,133.598064,71.082518
max,2023.000000,5.004279e+07,9.545158e+07,190.739913,71.082518


In [9]:
# Removing odd values, where we have more than 100% of the population displaced
df = df[df['total_idp_over_pop'] < 100]

In [10]:
# do a log transformation of the target variable
df['internally_displaced_persons'] = np.log1p(df['internally_displaced_persons'])

In [11]:
df.internally_displaced_persons

340    13.513171
380    14.857745
459    15.461179
490    15.525970
521    15.538416
         ...    
655    11.237185
769    11.566457
795    10.977619
822    11.716087
838    11.732912
Name: internally_displaced_persons, Length: 909, dtype: float64

In [12]:
env_factors = [
'ag.lnd.frst.k2', 'ag.lnd.prcp.mm',
       'ag.lnd.totl.k2', 'ag.srf.totl.k2', 'eg.cft.accs.ru.zs',
       'eg.cft.accs.ur.zs', 'eg.cft.accs.zs', 'eg.egy.prim.pp.kd',
       'eg.elc.accs.zs', 'eg.fec.rnew.zs', 'en.ghg.all.mt.ce.ar5',
       'en.ghg.ch4.ag.mt.ce.ar5', 'en.ghg.ch4.bu.mt.ce.ar5',
       'en.ghg.ch4.fe.mt.ce.ar5', 'en.ghg.ch4.ic.mt.ce.ar5',
       'en.ghg.ch4.mt.ce.ar5', 'en.ghg.ch4.pi.mt.ce.ar5',
       'en.ghg.ch4.tr.mt.ce.ar5', 'en.ghg.ch4.wa.mt.ce.ar5',
       'en.ghg.co2.bu.mt.ce.ar5', 'en.ghg.co2.ic.mt.ce.ar5',
       'en.ghg.co2.ip.mt.ce.ar5', 'en.ghg.co2.lu.mt.ce.ar5',
       'en.ghg.co2.mt.ce.ar5', 'en.ghg.co2.pi.mt.ce.ar5',
       'en.ghg.co2.tr.mt.ce.ar5', 'en.ghg.fgas.ip.mt.ce.ar5',
       'en.ghg.n2o.ag.mt.ce.ar5', 'en.ghg.n2o.bu.mt.ce.ar5',
       'en.ghg.n2o.fe.mt.ce.ar5', 'en.ghg.n2o.ic.mt.ce.ar5',
       'en.ghg.n2o.ip.mt.ce.ar5', 'en.ghg.n2o.mt.ce.ar5',
       'en.ghg.n2o.pi.mt.ce.ar5', 'en.ghg.n2o.tr.mt.ce.ar5',
       'en.ghg.n2o.wa.mt.ce.ar5', 'er.fsh.aqua.mt', 'er.fsh.capt.mt',
       'er.fsh.prod.mt', 'er.h2o.fwtl.k3', 'er.h2o.intr.k3', 'sp.pop.totl',
       'cpi_value', 'total_affected']

#impute mean to nas in env_factors

for col in env_factors:
       if col == 'total_affected':
              df[col].fillna(0, inplace=True)
       else:
              df[col].fillna(df[col].mean(), inplace=True)
df_clean = df
# Define independent (X) and dependent (y) variables
X = df_clean[env_factors]  # Environmental factors
y = df_clean["internally_displaced_persons"]  # Displacement

# Add a constant term for the regression model
X = sm.add_constant(X)

# Fit the regression model
model = sm.OLS(y, X).fit()

# Display model summary
model.summary()


/var/folders/p2/6x334wb90dvdf75sfntm2bxw0000gn/T/ipykernel_28204/1832782456.py:28: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mean(), inplace=True)
/var/folders/p2/6x334wb90dvdf75sfntm2bxw0000gn/T/ipykernel_28204/1832782456.py:26: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                 
========================================================================================
Dep. Variable:     internally_displaced_persons   R-squared:                       0.175
Model:                                      OLS   Adj. R-squared:                  0.133
Method:                           Least Squares   F-statistic:                     4.177
Date:                          Tue, 13 May 2025   Prob (F-statistic):           8.29e-17
Time:                                  20:59:24   Log-Likelihood:                -1820.9
No. Observations:                           909   AIC:                             3732.
Df Residuals:                               864   BIC:                             3948.
Df Model:                                    44                                         
Covariance Type:                      nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                       15.4727      1.528     10.129      0.000      12.474      18.471
ag.lnd.frst.k2            1.135e-05   2.87e-06      3.956      0.000    5.72e-06     1.7e-05
ag.lnd.prcp.mm              -0.0004      0.001     -0.600      0.548      -0.002       0.001
ag.lnd.totl.k2           -3.204e-06   2.08e-05     -0.154      0.878   -4.41e-05    3.77e-05
ag.srf.totl.k2            1.494e-06   2.05e-05      0.073      0.942   -3.87e-05    4.17e-05
eg.cft.accs.ru.zs            0.0096      0.040      0.241      0.809      -0.069       0.088
eg.cft.accs.ur.zs           -0.0108      0.018     -0.602      0.547      -0.046       0.024
eg.cft.accs.zs               0.0161      0.051      0.312      0.755      -0.085       0.117
eg.egy.prim.pp.kd           -0.2521      0.072     -3.484      0.001      -0.394      -0.110
eg.elc.accs.zs              -0.0112      0.010     -1.092      0.275      -0.031       0.009
eg.fec.rnew.zs              -0.0062      0.012     -0.509      0.611      -0.030       0.018
en.ghg.all.mt.ce.ar5         0.0025      0.001      2.646      0.008       0.001       0.004
en.ghg.ch4.ag.mt.ce.ar5   8.458e-05      0.001      0.073      0.942      -0.002       0.002
en.ghg.ch4.bu.mt.ce.ar5      0.0003      0.001      0.528      0.598      -0.001       0.001
en.ghg.ch4.fe.mt.ce.ar5      0.0017      0.001      1.738      0.083      -0.000       0.004
en.ghg.ch4.ic.mt.ce.ar5     -0.0002      0.002     -0.086      0.931      -0.004       0.004
en.ghg.ch4.mt.ce.ar5        -0.0010      0.001     -1.054      0.292      -0.003       0.001
en.ghg.ch4.pi.mt.ce.ar5     -0.0027      0.001     -2.112      0.035      -0.005      -0.000
en.ghg.ch4.tr.mt.ce.ar5   7.081e-05      0.001      0.108      0.914      -0.001       0.001
en.ghg.ch4.wa.mt.ce.ar5     -0.0003      0.001     -0.367      0.714      -0.002       0.001
en.ghg.co2.bu.mt.ce.ar5      0.0005      0.001      0.685      0.493      -0.001       0.002
en.ghg.co2.ic.mt.ce.ar5      0.0004      0.001      0.769      0.442      -0.001       0.001
en.ghg.co2.ip.mt.ce.ar5     -0.0006      0.001     -1.008      0.314      -0.002       0.001
en.ghg.co2.lu.mt.ce.ar5     -0.0002      0.001     -0.255      0.798      -0.002       0.001
en.ghg.co2.mt.ce.ar5        -0.0004      0.001     -0.302      0.762      -0.003       0.002
en.ghg.co2.pi.mt.ce.ar5     -0.0004      0.001     -0.756      0.450      -0.002       0.001
en.ghg.co2.tr.mt.ce.ar5   5.299e-06      0.001      0.004      0.997      -0.003       0.003
en.ghg.fgas.ip.mt.ce.ar5    -0.0009      0.001     -1.386      0.166      -0.002       0.000
en.ghg.n2o.ag.mt.ce.ar5      0.0023      0.002      1.155      0.248      -0.002       0.0

#### OLS Regression

In [13]:
# Split data into training (80%) and testing (20%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the regression model on the training set
model_train = sm.OLS(y_train, X_train).fit()

# Predict on the test set
y_pred = model_train.predict(X_test)

# Evaluate model performance
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

mae, r2

(1.2467414608805198, 0.07176022570106966)

#### Cross Validation for the OLS Model:
- First intent with K fold
- Note for fixing:  Do cross validation in the proportion of affected / total

In [14]:
# 1. Set up K-Fold cross-validation
k = 10
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# 2. Initialize lists to store metrics
mae_scores = []
r2_scores = []

# 3. Perform manual cross-validation with statsmodels
for train_index, test_index in kf.split(X):
    # Split data for this fold
    X_train_fold, X_test_fold = X.iloc[train_index], X.iloc[test_index]
    y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]
    
    # Train statsmodels OLS
    model_fold = sm.OLS(y_train_fold, X_train_fold).fit()
    
    # Make predictions
    y_pred_fold = model_fold.predict(X_test_fold)
    
    # Calculate and store performance metrics
    mae = mean_absolute_error(y_test_fold, y_pred_fold)
    r2 = r2_score(y_test_fold, y_pred_fold)
    
    mae_scores.append(mae)
    r2_scores.append(r2)

# 4. Calculate average performance
avg_mae = np.mean(mae_scores)
avg_r2 = np.mean(r2_scores)
std_mae = np.std(mae_scores)
std_r2 = np.std(r2_scores)

print(f"Cross-validation MAE: {avg_mae:.4f} ± {std_mae:.4f}")
print(f"Cross-validation R²: {avg_r2:.4f} ± {std_r2:.4f}")

Cross-validation MAE: 1.3949 ± 0.1687
Cross-validation R²: 0.0898 ± 0.0706


#### Random Forest Model

In [15]:
# Convert categorical variables to numerical using one-hot encoding
df_encoded = pd.get_dummies(df_clean.drop(columns=["internally_displaced_persons"]), drop_first=True)

# Fill missing values with the median
#df_encoded = df_encoded.fillna(df_encoded.median(numeric_only=True))
#y_filled = df["internally_displaced_persons"]

# Split into train and test sets
X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(df_encoded[env_factors], y, test_size=0.2, random_state=42)

# Initialize and train the Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train_rf, y_train_rf)

# Predict on test set
y_pred_rf = rf_model.predict(X_test_rf)

# Evaluate model performance
mae_rf = mean_absolute_error(y_test_rf, y_pred_rf)
r2_rf = r2_score(y_test_rf, y_pred_rf)

mae_rf, r2_rf

(0.6268454693874563, 0.6480114715788236)

#### Cross Validation for the Random Forest Model

In [16]:
# Setup K-fold cross-validation
k = 5  # adjust?? i think this is already pretty solid
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# Initialize the Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Prepare data
X = df_encoded[env_factors]
y = y

# Initialize lists to store performance metrics
mae_scores = []
r2_scores = []

# Perform cross-validation
for train_index, test_index in kf.split(X):
    # Split data for this fold
    X_train_fold, X_test_fold = X.iloc[train_index], X.iloc[test_index]
    y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]
    
    # Train model
    rf_model.fit(X_train_fold, y_train_fold)
    
    # Make predictions
    y_pred_fold = rf_model.predict(X_test_fold)
    
    # Calculate and store performance metrics
    mae = mean_absolute_error(y_test_fold, y_pred_fold)
    r2 = r2_score(y_test_fold, y_pred_fold)
    
    mae_scores.append(mae)
    r2_scores.append(r2)

# Calculate average performance
avg_mae = np.mean(mae_scores)
avg_r2 = np.mean(r2_scores)
std_mae = np.std(mae_scores)
std_r2 = np.std(r2_scores)

print(f"Cross-validation MAE: {avg_mae:.4f} ± {std_mae:.4f}")
print(f"Cross-validation R²: {avg_r2:.4f} ± {std_r2:.4f}")

Cross-validation MAE: 0.7676 ± 0.0858
Cross-validation R²: 0.6063 ± 0.0457


In [17]:
# Ridge doesn't take NAs, so we nede to impute them. The highest NAs we have are in variables that are about climate catastrophes, 
# where where we have them it means that there was NO climate catastrophe, so we can impute a 0 to them.

df_encoded.isna().sum().sort_values(ascending=False).head(20)
# impute a 0 to all the nas in the encoded df
df_encoded = df_encoded.fillna(0)

# Prepare data
X = df_encoded[env_factors]
y = y

#### Cross Validation for Ridge

In [18]:
# Create a pipeline that first standardizes, then applies Ridge regression
ridge_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('ridge', Ridge(alpha=1.0))
])

# Create scorers
mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)
r2_scorer = make_scorer(r2_score)

# Run cross-validation with the pipeline
ridge_mae_scores = -cross_val_score(ridge_pipeline, X, y, cv=5, scoring=mae_scorer)
ridge_r2_scores = cross_val_score(ridge_pipeline, X, y, cv=5, scoring=r2_scorer)

print("Ridge Cross-validation Results (with Pipeline):")
print(f"Cross-validation MAE: {ridge_mae_scores.mean():.4f} ± {ridge_mae_scores.std():.4f}")
print(f"Cross-validation R²: {ridge_r2_scores.mean():.4f} ± {ridge_r2_scores.std():.4f}")

Ridge Cross-validation Results (with Pipeline):
Cross-validation MAE: 1.5475 ± 0.3219
Cross-validation R²: -0.2671 ± 0.3552


In [26]:
# Lasso regression
lasso_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('lasso', Lasso(alpha=0.1))
])
# scorers
lasso_mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)
lasso_r2_scorer = make_scorer(r2_score)

# Run cross-validation with the pipeline
lasso_mae_scores = -cross_val_score(lasso_pipeline, X, y, cv=5, scoring=mae_scorer)
lasso_r2_scores = cross_val_score(lasso_pipeline, X, y, cv=5, scoring=r2_scorer)

print("Lasso Cross-validation Results (with Pipeline):")
print(f"Cross-validation MAE: {lasso_mae_scores.mean():.4f} ± {lasso_mae_scores.std():.4f}")
print(f"Cross-validation R²: {lasso_r2_scores.mean():.4f} ± {lasso_r2_scores.std():.4f}")



Lasso Cross-validation Results (with Pipeline):
Cross-validation MAE: 1.5447 ± 0.3829
Cross-validation R²: -0.2514 ± 0.2958


In [32]:
# make table comparing the models
results_log = pd.DataFrame({
    'Model': ['OLS', 'Random Forest', 'Ridge', 'Lasso'],
    'MAE (log target var)': [avg_mae, mae_rf, ridge_mae_scores.mean(), lasso_mae_scores.mean()],
    'R² (log target var)': [avg_r2, r2_rf, ridge_r2_scores.mean(), lasso_r2_scores.mean()]
})
# round the values to 2 decimal places
results_log['MAE (log target var)'] = results_log['MAE (log target var)'].round(2)
results_log['R² (log target var)'] = results_log['R² (log target var)'].round(2)
results_log

,Model,MAE (log target var),R² (log target var)
0,OLS,0.77,0.61
1,Random Forest,0.63,0.65
2,Ridge,1.55,-0.27
3,Lasso,1.54,-0.25


In [21]:
# transform results to original scale format
from sklearn.metrics import mean_absolute_error, r2_score
import numpy as np

# Invert the log1p transformation for actual values
y_test_ols_original = np.expm1(y_test)
y_test_rf_original = np.expm1(y_test_rf)

# Invert the log1p transformation for predictions
y_pred_ols_original = np.expm1(y_pred)
y_pred_rf_original = np.expm1(y_pred_rf)

# Calculate MAE and R² for OLS on the original scale
mae_ols_orig = mean_absolute_error(y_test_ols_original, y_pred_ols_original)
r2_ols_orig = r2_score(y_test_ols_original, y_pred_ols_original)

# Calculate MAE and R² for Random Forest on the original scale
mae_rf_orig = mean_absolute_error(y_test_rf_original, y_pred_rf_original)
r2_rf_orig = r2_score(y_test_rf_original, y_pred_rf_original)

# calculate mae and r2 for ridge model -> Inverting the log transformation for Ridge predictions (average scores)
mae_ridge_orig = np.expm1(ridge_mae_scores).mean()
r2_ridge_orig = ridge_r2_scores.mean()

# calculate mae and r2 for lasso model -> Inverting the log transformation for Lasso predictions (average scores)
mae_lasso_orig = np.expm1(lasso_mae_scores).mean()
r2_lasso_orig = lasso_r2_scores.mean()


# Print the results
print("OLS on original scale - MAE:", mae_ols_orig, "R²:", r2_ols_orig)
print("Random Forest on original scale - MAE:", mae_rf_orig, "R²:", r2_rf_orig)
print("Ridge on original scale - MAE:", mae_ridge_orig, "R²:", r2_ridge_orig)
print("Lasso on original scale - MAE:", mae_lasso_orig, "R²:", r2_lasso_orig)

OLS on original scale - MAE: 2403479.6446656203 R²: -0.47460066830953873
Random Forest on original scale - MAE: 996743.7738218033 R²: 0.6337732180409965
Ridge on original scale - MAE: 3.9615583873431914 R²: -0.2671348932776888
Lasso on original scale - MAE: 4.048579352072684 R²: -0.25139127092336844


In [33]:
# Create a DataFrame to present the results for all three models in original scale
results_original = pd.DataFrame({
    'Model': ['OLS', 'Random Forest', 'Ridge', 'Lasso'],
    'MAE (original scale)': [mae_ols_orig, mae_rf_orig, mae_ridge_orig, mae_lasso_orig],
    'R² (original scale)': [r2_ols_orig, r2_rf_orig, r2_ridge_orig, r2_lasso_orig]
})
# Round the values to 2 decimal places
results_original['MAE (original scale)'] = results_original['MAE (original scale)'].round(2)
results_original['R² (original scale)'] = results_original['R² (original scale)'].round(2)
results_original

,Model,MAE (original scale),R² (original scale)
0,OLS,2403479.64,-0.47
1,Random Forest,996743.77,0.63
2,Ridge,3.96,-0.27
3,Lasso,4.05,-0.25


In [34]:
# compare results of modles: first log models in original scale and the original models without log transformation
results_comparison = pd.merge(results_log, results_original, on='Model', suffixes=('_log', '_original'))
results_comparison

,Model,MAE (log target var),R² (log target var),MAE (original scale),R² (original scale)
0,OLS,0.77,0.61,2403479.64,-0.47
1,Random Forest,0.63,0.65,996743.77,0.63
2,Ridge,1.55,-0.27,3.96,-0.27
3,Lasso,1.54,-0.25,4.05,-0.25
